In [15]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks
  
print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, \
    NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D


<module 'tensorflow._api.v2.version' from '/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>
Num GPUs Available:  1


In [16]:
batch_size = 16
img_size = [240, 320]

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
df['filename'] = df["image_id"].astype(str) + ".png"

df = df.sample(frac=1).reset_index(drop=True)


print(df)


# new_df = pd.read_csv(r'machine-learning-in-science-ii-2023/new_training_norm.csv')
# new_df['filename'] = new_df["image_id"].astype(str) + "_speed-" + new_df["speed"].astype(str) + "_angle-" + new_df["angle"].astype(str) + ".png"
# #new_df['filename'] = new_df["image_id"].astype(str) + ".png"

#print(new_df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# new_x_train = new_df[0:int(len(new_df) * 0.6)]
# new_x_validate = new_df[int(len(new_df) * 0.6):int(len(new_df) * 0.8)]
# new_x_evaluate = new_df[int(len(new_df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0          2928  0.6875    0.0   2928.png
1          9065  0.6250    0.0   9065.png
2          8643  0.4375    1.0   8643.png
3          8447  0.6250    1.0   8447.png
4         10602  0.5000    1.0  10602.png
...         ...     ...    ...        ...
14821     11037  0.5625    0.0  11037.png
14822     17484  0.5000    0.0  17484.png
14823     16074  0.4375    1.0  16074.png
14824      6352  0.8750    1.0   6352.png
14825      3614  0.6875    1.0   3614.png

[14826 rows x 4 columns]


In [17]:

shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

#print(train_generator)

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')


Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.


In [18]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-1]:
    layer.trainable = False

#vgg.trainable = False # freeze the first layers to the imagenet weights

# CNN base
model = models.Sequential()

# model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=shape))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

model.add(transfer)
model.build()
        
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
#model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

#model.add(layers.Dropout(0.2))
model.add(layers.Flatten()),

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(32, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))

In [19]:
model.build()


In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_4 (Conv2D)           (None, 6, 8, 32)          368672    
                                                                 
 batch_normalization_12 (Bat  (None, 6, 8, 32)         128       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 6, 64)          18496     
                                                                 
 batch_normalization_13 (Bat  (None, 4, 6, 64)         256       
 chNormalization)                                                
                                                      

In [21]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0015,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100
556/556 [==============================] - 111s 191ms/step - loss: 0.1137 - val_loss: 0.0316
Epoch 2/100
556/556 [==============================] - 97s 174ms/step - loss: 0.0461 - val_loss: 0.0208
Epoch 3/100
556/556 [==============================] - 100s 180ms/step - loss: 0.0327 - val_loss: 0.0163
Epoch 4/100
556/556 [==============================] - 98s 177ms/step - loss: 0.0258 - val_loss: 0.0148
Epoch 5/100
556/556 [==============================] - 100s 179ms/step - loss: 0.0242 - val_loss: 0.0162
Epoch 6/100
556/556 [==============================] - 103s 184ms/step - loss: 0.0235 - val_loss: 0.0119
Epoch 7/100
556/556 [==============================] - 98s 176ms/step - loss: 0.0211 - val_loss: 0.0120
Epoch 8/100
556/556 [==============================] - 108s 195ms/step - loss: 0.0195 - val_loss: 0.0112
Epoch 9/100
556/556 [==============================] - 113s 202ms/step - loss: 0.0193 - val_loss: 0.0134
Epoch 10/100
556/556 [==============================] - 10

In [22]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)


test_datagen = ImageDataGenerator(rescale = 1./255)

test_images = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

print(test_images)

prediction = model.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.
32/32 [==============================] - 7s 119ms/step
(1020, 2)


In [23]:
from sklearn.metrics import classification_report

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)


prediction = model.predict(test_generator, verbose =1)

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('evaluate.csv')



93/93 [==============================] - 9s 99ms/step - loss: 0.0090
Found 2966 validated image filenames.
93/93 [==============================] - 8s 89ms/step


In [24]:
diff = abs(x_evaluate["angle"] - a)

results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":a, "Difference":diff})
results

#results.to_csv('evaluate.csv')

,Train,Predicted,Difference
11860,0.5000,0.479235,0.020765
11861,0.1250,0.151718,0.026718
11862,0.7500,0.721668,0.028332
11863,0.6875,0.577310,0.110190
11864,0.6875,0.724584,0.037084
...,...,...,...
14821,0.5625,0.604882,0.042382
14822,0.5000,0.522666,0.022666
14823,0.4375,0.489552,0.052052
14824,0.8750,0.727831,0.147169


In [25]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":b})
results

#results.to_csv('evaluate.csv')

,Train,Predicted
11860,1.0,1.0
11861,1.0,1.0
11862,1.0,1.0
11863,0.0,0.0
11864,1.0,1.0
...,...,...
14821,0.0,0.0
14822,0.0,0.0
14823,1.0,1.0
14824,1.0,1.0


In [26]:
model.save("model.h5")

In [27]:
def create_CNN_model(input_shape, hidden_layers, pretrained_model=None, num_non_trainable_layers=1,
                     output_layer={'BC': [1, 'sigmoid', 'binary_crossentropy'],
                                   'MC': [17, 'softmax', 'categorical_crossentropy']},
                     init='normal', optimize='adam', metrics=['accuracy', 'mse']):
    if pretrained_model:
        if pretrained_model == 'vgg16':
            base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'vgg19':
            base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'resnet50':
            base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'inceptionv3':
            base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'inceptionresnetv2':
            base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'xception':
            base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'mobilenet':
            base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'densenet':
            base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'nasnet':
            base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=input_shape)
        elif pretrained_model == 'efficientnet':
            base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        else:
            raise ValueError('Invalid pretrain_model parameter. Please select from vgg16, vgg19, resnet50, '
                             'inceptionv3, inceptionresnetv2, xception, mobilenet, densenet, nasnet, or efficientnet')

        # Set layers to be non-trainable
        for layer in base_model.layers[:-num_non_trainable_layers]:
            layer.trainable = False

        # create model
        model = Sequential()
        model.add(base_model)

    else:
        # create model
        model = Sequential()

    # print(hidden_layers)
    k = 3
    s = 1
    p = 2
    r = 0.2

    for i, layer in enumerate(hidden_layers):
        #         print(layer)
        #         print(type(layer))
        layer = str(layer)
        if i == 0:
            if '_' in layer:
                params = layer.split('_')
                for i, param in enumerate(params):
                    if i == 0:
                        filters = int("".join(filter(str.isdigit, param)))
                    #                     print(filters)
                    elif i == 1:
                        k = int("".join(filter(str.isdigit, param)))
                    #                     print(k)
                    elif i == 2:
                        s = int("".join(filter(str.isdigit, param)))
            #                     print(s)
            else:
                filters = int("".join(filter(str.isdigit, layer)))
            #               print(filters)

            model.add(Conv2D(filters, kernel_size=(k, k), strides=(s, s), activation='relu', input_shape=input_shape))

        elif 'C' in layer:
            if '_' in layer:
                params = layer.split('_')
                for i, param in enumerate(params):
                    if i == 0:
                        filters = int("".join(filter(str.isdigit, param)))
                    #                     print(filters)
                    elif i == 1:
                        k = int("".join(filter(str.isdigit, param)))
                    #                     print(k)
                    elif i == 2:
                        s = int("".join(filter(str.isdigit, param)))
            #                     print(s)
            else:
                filters = int("".join(filter(str.isdigit, layer)))
            #               print(filters)

            model.add(Conv2D(filters, (k, k), strides=(s, s), activation='relu'))


        elif 'MP' in layer:
            stride_flag = False
            if '_' in layer:
                params = layer.split('_')
                for i, param in enumerate(params):
                    if i == 1:
                        p = int("".join(filter(str.isdigit, param)))
                    #                     print(p)
                    elif i == 2:
                        s = int("".join(filter(str.isdigit, param)))
                        stride_flag = True
            #                     print(s)

            model.add(MaxPooling2D(pool_size=(p, p), strides=(s, s) if stride_flag else None))

        elif 'BN' in layer:

            model.add(BatchNormalization())


        elif 'F' in layer:

            model.add(Flatten())
        #             print('F')

        elif 'D' in layer:
            if '_' in layer:
                params = layer.split('_')
                for i, param in enumerate(params):
                    if i == 1:
                        r = int("".join(filter(str.isdigit, param)))

            model.add(Dropout(r))

        else:

            model.add(Dense(int(layer), kernel_initializer=init, activation='relu'))

    loss_functions = []
    for layer, params in output_layer.items():
        model.add(Dense(params[0], activation=params[1], kernel_initializer=init))
        loss_functions.append(params[2])

    model.compile(loss=loss_functions, optimizer=optimize, metrics=metrics)

    return model
